# 데이터 확인

In [1]:
import os

base_dir = './data'

# 1. action 디렉토리 내 json 파일 이름 (확장자 제거)
action_json_names = set()
action_path = os.path.join(base_dir, 'action')

for cls in os.listdir(action_path):
    cls_path = os.path.join(action_path, cls)
    if not os.path.isdir(cls_path): continue
    for session in os.listdir(cls_path):
        session_path = os.path.join(cls_path, session)
        if not os.path.isdir(session_path): continue
        for fname in os.listdir(session_path):
            if fname.endswith('.json'):
                json_name = os.path.splitext(fname)[0]
                action_json_names.add(json_name)

print(f"[ACTION] JSON 파일 수: {len(action_json_names)}")


[ACTION] JSON 파일 수: 829


In [2]:
# 2. emotion/situation 아래 폴더 이름 중 중복되는 것 확인
search_dirs = ['emotion', 'situation']
duplicate_folders = []

for top_dir in search_dirs:
    category_path = os.path.join(base_dir, top_dir)
    for cls in os.listdir(category_path):
        cls_path = os.path.join(category_path, cls)
        if not os.path.isdir(cls_path): continue

        for folder in os.listdir(cls_path):
            if folder in action_json_names:
                duplicate_folders.append((folder, top_dir, cls))

print(f"\n[중복된 폴더 이름 수]: {len(duplicate_folders)}")
for name, category, cls in duplicate_folders:
    print(f"- {name} (also in: {category}/{cls}/...)")


[중복된 폴더 이름 수]: 32
- cat-tailing-043157 (also in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-042680 (also in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-043196 (also in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-042787 (also in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-042917 (also in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-043158 (also in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-043151 (also in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-042699 (also in: emotion/EMOTION_불안_슬픔/...)
- cat-arch-032306 (also in: emotion/EMOTION_공포/...)
- 20201114_cat-arch-000925.mp4 (also in: emotion/EMOTION_공포/...)
- 20201113_cat-arch-000157.mp4 (also in: emotion/EMOTION_공포/...)
- cat-arch-025209 (also in: emotion/EMOTION_공포/...)
- cat-arch-025455 (also in: emotion/EMOTION_공포/...)
- cat-arch-011926 (also in: emotion/EMOTION_공포/...)
- cat-arch-037560 (also in: emotion/EMOTION_공포/...)
- 20201125_cat-arch-000928.mp4 (also in: emotion/EMOTION_공포/...)
- cat-arch-073365 (also in: emotion/EMOTION_공포/...)
- cat-arch

In [3]:
search_dirs = ['emotion', 'situation']
unmatched_folders = []

for top_dir in search_dirs:
    category_path = os.path.join(base_dir, top_dir)
    for cls in os.listdir(category_path):
        cls_path = os.path.join(category_path, cls)
        if not os.path.isdir(cls_path): continue

        for folder in os.listdir(cls_path):
            if folder not in action_json_names:
                unmatched_folders.append((folder, top_dir, cls))

print(f"\n[JSON으로 참조되지 않는 폴더 수]: {len(unmatched_folders)}")
for name, category, cls in unmatched_folders[:20]:  # 너무 많을 경우 상위 20개만 출력
    print(f"- {name} (in: {category}/{cls}/...)")

if len(unmatched_folders) > 20:
    print(f"... 생략됨 (총 {len(unmatched_folders)}개)")


[JSON으로 참조되지 않는 폴더 수]: 259
- cat-tailing-033497 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-029636 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-032529 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-034448 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-041499 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-029870 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-036608 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-034828 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-040887 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-037548 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-043876 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-030422 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-035982 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-029630 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-039436 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-029353 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-034024 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-034748 (in: emotion/EMOTION_불안_슬픔/...)
- cat-tailing-0326

---

# Task별 라벨 확인

In [4]:
import os
import json

base_dir = './data'
action_path = os.path.join(base_dir, 'action')

situation_labels = set()
action_labels = set()
emotion_labels = set()

# action 디렉토리 순회
for cls in os.listdir(action_path):
    cls_path = os.path.join(action_path, cls)
    if not os.path.isdir(cls_path): continue

    for session in os.listdir(cls_path):
        session_path = os.path.join(cls_path, session)
        if not os.path.isdir(session_path): continue

        for fname in os.listdir(session_path):
            if fname.endswith('.json'):
                json_path = os.path.join(session_path, fname)
                try:
                    with open(json_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)

                        situation = data.get('metadata', {}).get('owner', {}).get('situation')
                        action = data.get('metadata', {}).get('inspect', {}).get('action')
                        emotion = data.get('metadata', {}).get('inspect', {}).get('emotion')

                        if situation: situation_labels.add(situation)
                        if action: action_labels.add(action)
                        if emotion: emotion_labels.add(emotion)

                except Exception as e:
                    print(f"파일 읽기 실패: {json_path} - {e}")

# 결과 출력
print(f"\n[SITUATION] 라벨 수: {len(situation_labels)}")
print(situation_labels)

print(f"\n[ACTION] 라벨 수: {len(action_labels)}")
print(action_labels)

print(f"\n[EMOTION] 라벨 수: {len(emotion_labels)}")
print(emotion_labels)



[SITUATION] 라벨 수: 17
{'보호자와 떨어질 때/혼자 남겨지거나 낯선장소에 있을 때', '밥그릇, 장난감과 같은 소유물을 만질 때', '산책이나 노즈워크 중', '기타', '산책 준비 또는 산책중일 때', '다른 동물을 보거나 낯선 사람을 만날 때 산책 나왔을 때', '다른 사람이나 동물을 만났을 때', '낯선 장소에 있거나 낯선 소리가 날 때', '먹을것, 장난감이 앞에 있을 때', '싫어하는 부위를 만질 때', '편안히 쓰다듬어 줄 때', '보호자가 집에 돌아왔을 때', '휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등)', '잠들기 전이나 같이 누워있을 때', '빗질/발톱깍기/목욕 등 위생관리를 할 때', '낯선 소리가 나거나 낯선 사람을 봤을 때', '낯선 소리가 났을 때'}

[ACTION] 라벨 수: 13
{'허리를 아치로 세우는 동작', '걷거나 달리는 동작', '머리를 들이대는 동작', '발을 숨기고 웅크리고 앉는 동작', '좌우로 뒹구는 동작', '납작 엎드리는 동작', '그루밍하는 동작', '배를 보여주는 동작', '허리를 아치로 세움', '앞발로 꾹꾹 누르는 동작', '옆으로 눕는 동작', '꼬리를 흔드는 동작', '앞발을 뻗어 휘적거리는 동작'}

[EMOTION] 라벨 수: 6
{'화남/불쾌', '편안/안정', '행복/즐거움', '공격성', '공포', '불안/슬픔'}


In [6]:
import os
import json
from collections import Counter

base_dir = './data'
action_path = os.path.join(base_dir, 'action')

situation_counter = Counter()
action_counter = Counter()
emotion_counter = Counter()

# JSON 순회
for cls in os.listdir(action_path):
    cls_path = os.path.join(action_path, cls)
    if not os.path.isdir(cls_path): continue

    for session in os.listdir(cls_path):
        session_path = os.path.join(cls_path, session)
        if not os.path.isdir(session_path): continue

        for fname in os.listdir(session_path):
            if fname.endswith('.json'):
                json_path = os.path.join(session_path, fname)
                try:
                    with open(json_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)

                        situation = data.get('metadata', {}).get('owner', {}).get('situation')
                        action = data.get('metadata', {}).get('inspect', {}).get('action')
                        emotion = data.get('metadata', {}).get('inspect', {}).get('emotion')

                        if situation: situation_counter[situation] += 1
                        if action: action_counter[action] += 1
                        if emotion: emotion_counter[emotion] += 1

                except Exception as e:
                    print(f"파일 읽기 실패: {json_path} - {e}")

# 결과 출력 함수
def print_sorted_counter(title, counter):
    print(f"\n[{title}] 라벨 수: {len(counter)}")
    for label, count in counter.most_common():
        print(f"{label} : {count}회")

# 출력
print_sorted_counter("SITUATION", situation_counter)
print_sorted_counter("ACTION", action_counter)
print_sorted_counter("EMOTION", emotion_counter)



[SITUATION] 라벨 수: 17
기타 : 209회
휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등) : 185회
먹을것, 장난감이 앞에 있을 때 : 148회
잠들기 전이나 같이 누워있을 때 : 101회
편안히 쓰다듬어 줄 때 : 65회
보호자가 집에 돌아왔을 때 : 49회
낯선 소리가 났을 때 : 11회
빗질/발톱깍기/목욕 등 위생관리를 할 때 : 9회
싫어하는 부위를 만질 때 : 9회
산책이나 노즈워크 중 : 9회
밥그릇, 장난감과 같은 소유물을 만질 때 : 8회
낯선 소리가 나거나 낯선 사람을 봤을 때 : 7회
산책 준비 또는 산책중일 때 : 5회
다른 동물을 보거나 낯선 사람을 만날 때 산책 나왔을 때 : 5회
다른 사람이나 동물을 만났을 때 : 4회
낯선 장소에 있거나 낯선 소리가 날 때 : 4회
보호자와 떨어질 때/혼자 남겨지거나 낯선장소에 있을 때 : 1회

[ACTION] 라벨 수: 13
그루밍하는 동작 : 187회
허리를 아치로 세우는 동작 : 175회
꼬리를 흔드는 동작 : 49회
앞발을 뻗어 휘적거리는 동작 : 48회
걷거나 달리는 동작 : 48회
납작 엎드리는 동작 : 47회
배를 보여주는 동작 : 46회
옆으로 눕는 동작 : 46회
좌우로 뒹구는 동작 : 45회
머리를 들이대는 동작 : 45회
앞발로 꾹꾹 누르는 동작 : 44회
발을 숨기고 웅크리고 앉는 동작 : 44회
허리를 아치로 세움 : 5회

[EMOTION] 라벨 수: 6
편안/안정 : 586회
행복/즐거움 : 124회
공격성 : 51회
화남/불쾌 : 30회
불안/슬픔 : 21회
공포 : 17회


# TASK별 라벨들의 조합 확인하기

In [77]:
import os
import json
from collections import Counter

base_dir = './data'
action_path = os.path.join(base_dir, 'action')

combo_counter = Counter()

for cls in os.listdir(action_path):
    cls_path = os.path.join(action_path, cls)
    if not os.path.isdir(cls_path): continue

    for session in os.listdir(cls_path):
        session_path = os.path.join(cls_path, session)
        if not os.path.isdir(session_path): continue

        for fname in os.listdir(session_path):
            if fname.endswith('.json'):
                json_path = os.path.join(session_path, fname)
                try:
                    with open(json_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)

                        action = data.get('metadata', {}).get('inspect', {}).get('action')
                        emotion = data.get('metadata', {}).get('inspect', {}).get('emotion')
                        situation = data.get('metadata', {}).get('owner', {}).get('situation')

                        if action and emotion and situation:
                            combo = (action, emotion, situation)
                            combo_counter[combo] += 1

                except Exception as e:
                    print(f"파일 읽기 실패: {json_path} - {e}")

# 조합 출력
print(f"\n[조합 총 개수] : {len(combo_counter)}")
# for combo, count in combo_counter.most_common():
#     print(f"{combo} : {count}회")

import pandas as pd
df = pd.DataFrame(
    [(action, emotion, situation, count) for (action, emotion, situation), count in combo_counter.items()],
    columns=["action", "emotion", "situation", "frequency"]
)


[조합 총 개수] : 163


In [78]:
df

,action,emotion,situation,frequency
0,그루밍하는 동작,편안/안정,"휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등)",94
1,그루밍하는 동작,편안/안정,잠들기 전이나 같이 누워있을 때,29
2,그루밍하는 동작,편안/안정,보호자가 집에 돌아왔을 때,3
3,그루밍하는 동작,편안/안정,기타,33
4,그루밍하는 동작,편안/안정,편안히 쓰다듬어 줄 때,16
...,...,...,...,...
158,납작 엎드리는 동작,편안/안정,싫어하는 부위를 만질 때,1
159,납작 엎드리는 동작,화남/불쾌,빗질/발톱깍기/목욕 등 위생관리를 할 때,1
160,납작 엎드리는 동작,공포,낯선 소리가 났을 때,1
161,납작 엎드리는 동작,편안/안정,편안히 쓰다듬어 줄 때,2


In [79]:
print(df['frequency'].describe())

count    163.000000
mean       5.085890
std        9.309758
min        1.000000
25%        1.000000
50%        2.000000
75%        5.000000
max       94.000000
Name: frequency, dtype: float64


In [80]:
print("최대값:", df['frequency'].max())
print("최소값:", df['frequency'].min())
print("평균값:", df['frequency'].mean())
print("중앙값:", df['frequency'].median())

최대값: 94
최소값: 1
평균값: 5.085889570552148
중앙값: 2.0


---

# 멀티 라벨 태스크를 위한 데이터셋 구성

In [81]:
import os
import json
import random
import pandas as pd
import shutil

# 조합 빈도 DataFrame 로드
combo_df = df  # action, emotion, situation, frequency

# 기준 빈도 설정
FREQ_THRESHOLD = 5

base_dir = "./data/action"  # 실제 data/action 경로에 맞게 변경

result_rows = []

for action_dir in os.listdir(base_dir):
    action_dir_path = os.path.join(base_dir, action_dir)
    if not os.path.isdir(action_dir_path):
        continue

    for video_folder in os.listdir(action_dir_path):
        video_path = os.path.join(action_dir_path, video_folder)
        if not os.path.isdir(video_path):
            continue

        json_files = [f for f in os.listdir(video_path) if f.endswith(".json")]
        if not json_files:
            continue
        
        json_path = os.path.join(video_path, json_files[0])
        with open(json_path, "r", encoding="utf-8") as f:
            meta = json.load(f)
        
        # json에서 action, emotion, situation 추출
        action = meta["metadata"]["inspect"].get("action")
        emotion = meta["metadata"]["inspect"].get("emotion")
        situation = meta["metadata"]["owner"].get("situation")

        if not (action and emotion and situation):
            continue

        # 빈도수 확인
        freq_row = combo_df[
            (combo_df["action"] == action) &
            (combo_df["emotion"] == emotion) &
            (combo_df["situation"] == situation)
        ]

        frequency = int(freq_row["frequency"])

        # 이미지 파일 리스트
        image_files = sorted([f for f in os.listdir(video_path) if f.lower().endswith(".jpg")])
        if not image_files:
            continue

        if frequency > FREQ_THRESHOLD:
            # 빈도 높으면 랜덤 1개만 저장
            chosen_file = random.choice(image_files)
            image_path = os.path.join(video_path, chosen_file)
            result_rows.append({
                "image_path": image_path.strip(),
                "action": action.strip(),
                "emotion": emotion.strip(),
                "situation": situation.strip()
            })
        else:
            # 빈도 낮으면 모두 저장
            for img_file in image_files:
                image_path = os.path.join(video_path, img_file)
                result_rows.append({
                    "image_path": image_path.strip(),
                    "action": action.strip(),
                    "emotion": emotion.strip(),
                    "situation": situation.strip()
                })

# DataFrame 생성 후 CSV 저장
df_result = pd.DataFrame(result_rows)
df_result.to_csv("./data/selected_images.csv", index=False, encoding="utf-8-sig")

print("CSV 저장 완료. 총 이미지 수:", len(df_result))

/tmp/ipykernel_1425472/2804735429.py:50: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  frequency = int(freq_row["frequency"])


CSV 저장 완료. 총 이미지 수: 18172


In [85]:
import pandas as pd

# CSV 파일 불러오기
df_select = pd.read_csv("./data/selected_images.csv")

# 각 라벨별 분포 확인하기
print("Action 분포:")
print(df_select['action'].value_counts())

print("\nEmotion 분포:")
print(df_select['emotion'].value_counts())

print("\nSituation 분포:")
print(df_select['situation'].value_counts())


Action 분포:
action
꼬리를 흔드는 동작           3378
좌우로 뒹구는 동작           2616
납작 엎드리는 동작           2324
걷거나 달리는 동작           2009
머리를 들이대는 동작          1577
허리를 아치로 세우는 동작       1436
배를 보여주는 동작           1375
옆으로 눕는 동작             962
앞발로 꾹꾹 누르는 동작         700
앞발을 뻗어 휘적거리는 동작       636
그루밍하는 동작              532
발을 숨기고 웅크리고 앉는 동작     503
허리를 아치로 세움            124
Name: count, dtype: int64

Emotion 분포:
emotion
편안/안정     9194
행복/즐거움    4754
불안/슬픔     1629
화남/불쾌     1129
공격성        915
공포         551
Name: count, dtype: int64

Situation 분포:
situation
먹을것, 장난감이 앞에 있을 때                       4112
기타                                      3205
보호자가 집에 돌아왔을 때                          2905
편안히 쓰다듬어 줄 때                            2056
휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등)    1143
산책이나 노즈워크 중                              710
싫어하는 부위를 만질 때                            638
밥그릇, 장난감과 같은 소유물을 만질 때                   625
잠들기 전이나 같이 누워있을 때                        519
낯선 소리가 나거나 낯선 사람을 봤을 때                   488
낯선 소리가

In [86]:
# action, emotion, situation 조합별 개수 세기
combo_counts = df_select.groupby(['action', 'emotion', 'situation']).size().reset_index(name='frequency')

print(combo_counts)

             action emotion                             situation  frequency
0        걷거나 달리는 동작     공격성                                    기타        164
1        걷거나 달리는 동작     공격성                     먹을것, 장난감이 앞에 있을 때        164
2        걷거나 달리는 동작     공격성                        보호자가 집에 돌아왔을 때          3
3        걷거나 달리는 동작      공포                           낯선 소리가 났을 때        111
4        걷거나 달리는 동작   편안/안정                                    기타         14
..              ...     ...                                   ...        ...
158  허리를 아치로 세우는 동작   화남/불쾌  휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등)         25
159      허리를 아치로 세움      공포                           낯선 소리가 났을 때         49
160      허리를 아치로 세움  행복/즐거움                     먹을것, 장난감이 앞에 있을 때         25
161      허리를 아치로 세움   화남/불쾌                           낯선 소리가 났을 때         25
162      허리를 아치로 세움   화남/불쾌                 낯선 장소에 있거나 낯선 소리가 날 때         25

[163 rows x 4 columns]


---

In [89]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. 조합을 하나의 class로 봄
df_select["label"] = df_select[["action", "emotion", "situation"]].agg("||".join, axis=1)

# 2. 먼저 train_val, test 나누기 (예: test 15%)
train_val_df, test_df = train_test_split(
    df_select, test_size=0.15, stratify=df_select["label"], random_state=42
)

# 3. train, val 나누기 (예: val 15% of train_val)
train_df, val_df = train_test_split(
    train_val_df, test_size=0.15, stratify=train_val_df["label"], random_state=42
)

# 4. 확인
print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

# 5. label 컬럼 제거 (필요하다면)
for d in [train_df, val_df, test_df]:
    d.drop(columns=["label"], inplace=True)

# 6. 저장
train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)

Train: 13129 | Val: 2317 | Test: 2726


In [90]:
import pandas as pd

# 파일 불러오기
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")
test_df = pd.read_csv("test.csv")

# label 조합 생성
for df in [train_df, val_df, test_df]:
    df["label"] = df[["action", "emotion", "situation"]].agg("||".join, axis=1)

# 분포 확인 함수
def print_distribution(df, name):
    print(f"\n[{name} 분포] 총 샘플 수: {len(df)}")
    print(df["label"].value_counts().head(10))  # 상위 10개 조합 출력

# 각 분포 출력
print_distribution(train_df, "Train")
print_distribution(val_df, "Validation")
print_distribution(test_df, "Test")



[Train 분포] 총 샘플 수: 13129
label
좌우로 뒹구는 동작||편안/안정||보호자가 집에 돌아왔을 때        398
좌우로 뒹구는 동작||행복/즐거움||기타                   349
좌우로 뒹구는 동작||행복/즐거움||보호자가 집에 돌아왔을 때       335
꼬리를 흔드는 동작||편안/안정||기타                    326
앞발로 꾹꾹 누르는 동작||편안/안정||편안히 쓰다듬어 줄 때       306
납작 엎드리는 동작||편안/안정||잠들기 전이나 같이 누워있을 때     305
걷거나 달리는 동작||행복/즐거움||먹을것, 장난감이 앞에 있을 때    284
꼬리를 흔드는 동작||행복/즐거움||먹을것, 장난감이 앞에 있을 때    283
배를 보여주는 동작||편안/안정||먹을것, 장난감이 앞에 있을 때     270
납작 엎드리는 동작||편안/안정||기타                    260
Name: count, dtype: int64

[Validation 분포] 총 샘플 수: 2317
label
좌우로 뒹구는 동작||편안/안정||보호자가 집에 돌아왔을 때        70
좌우로 뒹구는 동작||행복/즐거움||기타                   62
좌우로 뒹구는 동작||행복/즐거움||보호자가 집에 돌아왔을 때       59
꼬리를 흔드는 동작||편안/안정||기타                    58
납작 엎드리는 동작||편안/안정||잠들기 전이나 같이 누워있을 때     54
앞발로 꾹꾹 누르는 동작||편안/안정||편안히 쓰다듬어 줄 때       54
꼬리를 흔드는 동작||행복/즐거움||먹을것, 장난감이 앞에 있을 때    50
걷거나 달리는 동작||행복/즐거움||먹을것, 장난감이 앞에 있을 때    50
배를 보여주는 동작||편안/안정||먹을것, 장난감이 앞에 있을 때     48
납작 엎드리는 동작||편안/안정||기타                    46
Name: count, dty